In [1]:
# This notebook do inference on a trained model

# NVIDIA-SMI 535.183.01 
# CUDA Version: 10.1
# CUDNN Version: 
    #define CUDNN_MAJOR 7
    #define CUDNN_MINOR 6
    #define CUDNN_PATCHLEVEL 5
# Tensorflow Version: 2.2.0
# Python Version: 3.8.10

#### Library

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Set pandas to display all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Suppress pandas performance warnings
import warnings
warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)
from datetime import datetime, timedelta
import joblib

In [3]:
# check tf version
print('TensorFlow version:', tf.__version__)

#  Check if TensorFlow can access GPUs
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"TensorFlow is using GPU: {gpus}")
else:
    print("No GPU found, using CPU instead.")

TensorFlow version: 2.2.0
TensorFlow is using GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2025-03-20 16:43:34.374644: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2025-03-20 16:43:34.421442: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-03-20 16:43:34.421759: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce 940MX computeCapability: 5.0
coreClock: 1.189GHz coreCount: 3 deviceMemorySize: 3.95GiB deviceMemoryBandwidth: 37.33GiB/s
2025-03-20 16:43:34.422256: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2025-03-20 16:43:34.430493: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2025-03-20 16:43:34.433425: I tensorflow/stream_executor/platform

#### Load Inference Data

In [4]:
# load inference data
inference_data = pd.read_csv('/home/kevin/Downloads/BESS/Jobs/Inferencing/data/raw/merged_df_cleaned.csv')

#### Model Inferencing

In [5]:
# Load the saved model
def load_model(save_path):
    model = tf.keras.models.load_model(save_path)
    print(f"Model loaded from {save_path}")
    return model

save_path = f"/home/kevin/Downloads/BESS/Jobs/Retraining/artifacts/lstm_model_{datetime.now().strftime('%Y%m%d')}.h5"

# load trained model
model = load_model(save_path)

2025-03-20 16:43:34.614507: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2025-03-20 16:43:34.653037: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 2699905000 Hz
2025-03-20 16:43:34.653605: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fdaec000b60 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2025-03-20 16:43:34.653648: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2025-03-20 16:43:34.721619: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-03-20 16:43:34.722010: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x38ec0e70 initialized for platform CUDA (this does not guarantee that XLA wil

Model loaded from /home/kevin/Downloads/BESS/Jobs/Retraining/artifacts/lstm_model_20250320.h5


In [6]:
# Load the scaler
def load_scaler(save_path):
    scaler = joblib.load(save_path)
    print(f"Scaler loaded from {save_path}")
    return scaler

save_path = f"/home/kevin/Downloads/BESS/Jobs/Retraining/artifacts/scaler_{datetime.now().strftime('%Y%m%d')}.pkl"

scaler = load_scaler(save_path)

Scaler loaded from /home/kevin/Downloads/BESS/Jobs/Retraining/artifacts/scaler_20250320.pkl


In [7]:
# data transformation
X_inference = inference_data.drop(columns=['datetime_'])
X_inference = scaler.transform(X_inference)
X_inference = X_inference.reshape((X_inference.shape[0], 1, int(X_inference.shape[1])))

# make predictions
y_inference_pred = model.predict(X_inference)
y_inference_pred = y_inference_pred.reshape(-1, 1)
y_inference_pred

2025-03-20 16:43:38.368627: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2025-03-20 16:43:38.693829: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7


array([[40.84008 ],
       [40.898457],
       [50.041996],
       [55.528152],
       [55.33215 ],
       [52.632683],
       [47.74548 ],
       [43.815575],
       [37.34431 ],
       [35.550556],
       [35.31981 ],
       [35.70145 ],
       [37.394756],
       [40.98639 ],
       [45.748547],
       [47.275097],
       [40.63331 ],
       [30.939926],
       [28.114107],
       [29.324333],
       [30.616375],
       [30.73483 ],
       [30.499538],
       [29.014   ]], dtype=float32)

In [8]:
# save the predictions
pred_df_new = pd.DataFrame()
pred_df_new['datetime_'] = inference_data['datetime_']
pred_df_new['predicted_pool_price'] = y_inference_pred

# load the existing predictions
try:
    pred_df = pd.read_csv('/home/kevin/Downloads/BESS/Jobs/Inferencing/data/predictions/pred_df.csv')
except:
    pred_df = pd.DataFrame()

# concatenate the new predictions with the existing predictions
pred_df = pd.concat([pred_df, pred_df_new], axis=0, ignore_index=True)
# drop duplicates
pred_df = pred_df.drop_duplicates(subset=['datetime_'], keep='last')
# sort the dataframe by datetime
pred_df = pred_df.sort_values(by='datetime_', ascending=True)
# reset the index
pred_df = pred_df.reset_index(drop=True)
# save the predictions
pred_df.to_csv('/home/kevin/Downloads/BESS/Jobs/Inferencing/data/predictions/pred_df.csv', index=False)